# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "../output_data/cities.csv"

df = pd.read_csv(output_data_file)
df
vacation_data_df = df.drop(columns=["City_ID"])
vacation_data_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,puerto ayora,-0.74,-90.35,80.01,87,97,8.01,EC,1587606423
1,ushuaia,-54.80,-68.30,39.20,64,100,8.05,AR,1587606510
2,jamestown,42.10,-79.24,32.00,81,90,8.05,US,1587606425
3,busselton,-33.65,115.33,70.00,62,94,5.84,AU,1587606517
4,georgiyevskoye,58.73,45.02,25.56,92,100,11.25,RU,1587606517
...,...,...,...,...,...,...,...,...,...
552,aasiaat,68.71,-52.87,34.50,84,100,5.79,GL,1587606631
553,kedzierzyn-kozle,50.35,18.23,41.00,70,0,1.01,PL,1587606632
554,san jeronimo,-11.95,-75.28,50.50,29,98,1.83,PE,1587606632
555,kutum,14.20,24.67,62.15,16,97,4.45,SD,1587606632


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = vacation_data_df[["Lat", "Lng"]]
humidity = vacation_data_df["Humidity"].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig




Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# A max temperature lower than 80 degrees but higher than 70
# Wind speed less than 10 mph
# Zero cloudiness
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal

weather_conditions_df = vacation_data_df[(vacation_data_df["Max Temp"] < 80) & (vacation_data_df["Max Temp"] > 70)
                        & (vacation_data_df["Wind Speed"] < 10) & (vacation_data_df["Cloudiness"]  == 0) ]



weather_conditions_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
124,villa bisono,19.56,-70.88,75.99,79,0,6.49,DO,1587606541
162,tambacounda,13.75,-13.76,79.57,27,0,4.88,SN,1587606550
193,tessalit,20.20,1.01,78.62,12,0,8.21,ML,1587606555
203,grand gaube,-20.01,57.66,73.99,80,0,1.99,MU,1587606558
257,cumpas,29.98,-109.78,71.89,18,0,6.11,MX,1587606569
269,aswan,24.09,32.91,74.44,20,0,9.55,EG,1587606571
389,riyadh,24.69,46.72,71.60,73,0,3.36,SA,1587606469
426,port macquarie,-31.43,152.92,77.00,51,0,1.01,AU,1587606602
460,susangerd,31.56,48.18,71.60,68,0,4.70,IR,1587606609
527,sainte-rose,16.33,-61.70,77.00,78,0,5.82,GP,1587606623


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = weather_conditions_df[["City","Country","Lat","Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df


,City,Country,Lat,Lng,Hotel Name
124,villa bisono,DO,19.56,-70.88,
162,tambacounda,SN,13.75,-13.76,
193,tessalit,ML,20.20,1.01,
203,grand gaube,MU,-20.01,57.66,
257,cumpas,MX,29.98,-109.78,
269,aswan,EG,24.09,32.91,
389,riyadh,SA,24.69,46.72,
426,port macquarie,AU,-31.43,152.92,
460,susangerd,IR,31.56,48.18,
527,sainte-rose,GP,16.33,-61.70,


In [15]:

params ={ "radius": 5000,
           "types": "lodging",
           "key": g_key
        }


base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for idx,row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params)
    hotels_nearby = response.json()
    try:
        hotel_df.loc[idx,"Hotel Name"] = hotels_nearby["results"][0]["name"]
    except (KeyError,IndexError):
        print("Hotel not found")
hotel_df

Hotel not found
Hotel not found


,City,Country,Lat,Lng,Hotel Name
124,villa bisono,DO,19.56,-70.88,BANCO ADEMI
162,tambacounda,SN,13.75,-13.76,
193,tessalit,ML,20.20,1.01,
203,grand gaube,MU,-20.01,57.66,Veranda Paul et Virginie Hotel & Spa
257,cumpas,MX,29.98,-109.78,Hotel La Palma
269,aswan,EG,24.09,32.91,Nuba Nile
389,riyadh,SA,24.69,46.72,InterContinental Riyadh
426,port macquarie,AU,-31.43,152.92,Rydges Port Macquarie
460,susangerd,IR,31.56,48.18,نانوایی حبیب
527,sainte-rose,GP,16.33,-61.70,Habitation du Comté Hôtel Restaurant


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)
# Add layer
fig.add_layer(marker_layer)
# Display figure
fig



Figure(layout=FigureLayout(height='420px'))